# 🔬 Fine-Tune microsoft/phi-2 mit LoRA auf Google Colab
Dieses Notebook lädt dein `train_data.csv` und trainiert `phi-2` mit PEFT/LoRA.

In [1]:
# ✅ GPU prüfen
!nvidia-smi

Sun May 18 15:51:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 📦 Installation
!pip install -q peft transformers accelerate bitsandbytes datasets

In [3]:
# 📁 CSV-Datei hochladen
from google.colab import files
uploaded = files.upload()  # Lade train_data.csv hoch

Saving train_data.csv to train_data (4).csv


In [4]:
# 📊 Daten laden
import pandas as pd
df = pd.read_csv('train_data.csv')
df.head()

,prompt,response
0,The risk of heart failure is 10%. Alcohol doub...,"base risk: 10%, new absolute risk: number not ..."
1,The risk of infection is 2%. Smoking triples t...,"base risk: 2%, new absolute risk: number not d..."
2,The chance of cancer is 5%. Medication reduces...,"base risk: 5%, new absolute risk: number not d..."
3,Heart attack risk is 8%. Exercise cuts it in h...,"base risk: 8%, new absolute risk: number not d..."
4,The risk of stroke is 6%. Alcohol increases th...,"base risk: 6%, new absolute risk: number not d..."


In [5]:
from huggingface_hub import login
login()

In [7]:
# 🧠 Modell laden & vorbereiten
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

model_id = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto'
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# 🔧 LoRA-Konfiguration
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)
model = get_peft_model(model, config)

In [9]:
# 📚 Dataset vorbereiten
from datasets import Dataset

dataset = Dataset.from_pandas(df[['prompt', 'response']])

def tokenize(example):
    model_input = tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=32
    )
    labels = tokenizer(
        example["response"],
        padding="max_length",
        truncation=True,
        max_length=32
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_ds = dataset.map(tokenize, batched=False)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [10]:
# 🚀 Training
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='mistral-lora-output',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    logging_steps=1,
    save_strategy='epoch',
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,10.383500
2,10.047000
3,8.914400
4,9.785900
5,8.907700
6,8.151000
7,8.525800
8,8.704800
9,8.499100
10,8.716200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=15, training_loss=8.738502470652262, metrics={'train_runtime': 22.3529, 'train_samples_per_second': 0.671, 'train_steps_per_second': 0.671, 'total_flos': 20488515747840.0, 'train_loss': 8.738502470652262, 'epoch': 3.0})

In [11]:
# 💾 Modell speichern
model.save_pretrained('mistral-lora-output')
tokenizer.save_pretrained('mistral-lora-output')

('mistral-lora-output/tokenizer_config.json',
 'mistral-lora-output/special_tokens_map.json',
 'mistral-lora-output/tokenizer.model',
 'mistral-lora-output/added_tokens.json',
 'mistral-lora-output/tokenizer.json')

In [12]:
from transformers import pipeline

# 📥 Prompt einsetzen
prompt = ": Please extract the following information, if possible and only if it is directly given in the sentence. " \
"Please dont do calculations on your own. The base risk, the new absolute risk, " \
"the relative risk. Name the name of the number and then the number itself. Remember, dont do calculations on your own. " \
"If the wanted number is not directly given, just answer with 'number not directly given'. " \
"Here is the sentence from which you have to extract the information: The risk of heart failure is 10%. Alcohol doubles this risk."

# 🚀 Modellpipeline verwenden (wenn `model` und `tokenizer` bereits definiert sind)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=torch.float16
)

# 📤 Ausgabe erzeugen
output = pipe(prompt, max_new_tokens=50, do_sample=False)

# 🖨️ Ergebnis anzeigen
print("🔍 Modellantwort:\n", output[0]["generated_text"])

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

🔍 Modellantwort:
 : Please extract the following information, if possible and only if it is directly given in the sentence. Please dont do calculations on your own. The base risk, the new absolute risk, the relative risk. Name the name of the number and then the number itself. Remember, dont do calculations on your own. If the wanted number is not directly given, just answer with 'number not directly given'. Here is the sentence from which you have to extract the information: The risk of heart failure is 10%. Alcohol doubles this risk.

1. Base risk:

2. New absolute risk:

3. Relative risk:

4. Number not directly given:

5. Number not directly given:

6. Number not directly given:
